In [1]:
# Import retro to play Street Fighter using a ROM
import retro
# Import time to slow down game
import time

import gym

import pandas as pd

# Import environment base class for a wrapper
from gym import Env
# Import the space shapes for the environment 
from gym.spaces import MultiBinary, Box
# import  numpy to calcualte frame delta
import numpy as np
# Import opencv for graysacaling
import cv2
# Import matplotlib for plotting the image 
from matplotlib import pyplot as plt

import stable_baselines3


# Importing the optimzation frame - HPO
import optuna
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common import env_checker
from stable_baselines3.common import utils as utils
# Import os to deal with filepaths
import os

# Import base callback 
from stable_baselines3.common.callbacks import BaseCallback

In [2]:
# Create custom environment 
class StreetFighter(Env): 
    def __init__(self):
        super().__init__()
        
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
            
    def reset(self):
        # Return the first frame 
        obs = self.game.reset()
        obs = self.preprocess(obs) 
        self.previous_frame = obs 
        
        # Create a attribute to hold the score delta 
        self.score = 0 
        self.health = 0
        self.enemy_health = 0
        self.matches_won = 0
        
        return obs
    
    def preprocess(self, observation): 
        # Grayscaling 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        # Resize 
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        # Add the channels value
        channels = np.reshape(resize, (84,84,1))
        return channels 
    
    def step(self, action):           
        
        # Take a step 
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)       
        
        # Frame delta 
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs 
       
        
        # Reward function:
                
        # # Earn reward for hurt component hp                      
        if info['health']!=176 & info['health']!=0 & info['health']!=-1:
            reward = (self.enemy_health - info['enemy_health'])*100 + (info['health'] - self.health)*100
            
        # # Earn negative reward for losing hp                      
        # if info['health']!=176 & info['health']!=0 & info['health']!=-1:
        #     reward = (info['health'] - self.health)*100
         
        if self.enemy_health<=-1:
            reward = 100000
            
        if self.health<=-1:
            reward = -100000
        
        self.health = info['health']
        self.enemy_health = info['enemy_health']
        self.matches_won = info['matches_won']
        self.score = info['score']
        
        return frame_delta, reward, done, info
    
    def render(self, *args, **kwargs):
        self.game.render()
        
    def close(self):
        self.game.close()

In [3]:

env = StreetFighter()

In [4]:
# obs = env.reset()

# done = False

# for game in range(1):
#     while not done:
#         if done:
#             obs = env.reset()
#         env.render()
#         obs, reward, done, info = env.step(env.action_space.sample())
#         time.sleep(0.00)
        
        
        
#         if reward !=0:
#             print(f"reward: {reward}")
#             print(info)
    


In [5]:
LOG_DIR = './rfd3_logs/'
OPT_DIR = './rfd3_opt/'

In [12]:
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [13]:
CHECKPOINT_DIR = './rfd3_train/'

In [14]:
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)

In [18]:
# Create environment 
env.close()
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [19]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [20]:
model.learn(total_timesteps=5000000, callback=callback)

Logging to ./rfd3_logs/PPO_1
-----------------------------
| time/              |      |
|    fps             | 513  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
--------------------------------------
| time/                   |          |
|    fps                  | 385      |
|    iterations           | 2        |
|    time_elapsed         | 10       |
|    total_timesteps      | 4096     |
| train/                  |          |
|    approx_kl            | 73.33855 |
|    clip_fraction        | 0.817    |
|    clip_range           | 0.2      |
|    entropy_loss         | -4.15    |
|    explained_variance   | -4.3e-05 |
|    learning_rate        | 0.0003   |
|    loss                 | 7.47e+04 |
|    n_updates            | 10       |
|    policy_gradient_loss | 0.156    |
|    value_loss           | 2.1e+05  |
--------------------------------------
---------------------------------------
| time/            